# Script Adapatation vs Non SVM classification (default parameters, k = linear, C = 1) 
## Sliding window, concatonate windows, seperate conditions : Riemannian w/out CSP, CSP + Riemann

In [1]:
from sklearnex import patch_sklearn

patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
import numpy as np
import scipy.io
import os

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.covariance import LedoitWolf

from pyriemann.estimation import Covariances
from pyriemann.classification import TSclassifier
from pyriemann.spatialfilters import CSP
from pyriemann.tangentspace import TangentSpace
#from pyriemann.classification import SVC

from sklearn.svm import SVC

import h5py

import time

In [3]:
start_time = time.time()

## Load datasets

In [3]:
LoadingPath='/home/nicole/Documents/AudioCueWalking_analysis/Variables/AdaptVsNon/Individual_Subjects/WindowingEEGSignals/TrainingTestingDatasets/SeperateConditions/Delay'

In [4]:
SubjectID=17 #subject = SubjectID + 1 

In [6]:
n_components = 108  

### Load : TrainingDataset & TestingDataset

In [5]:
TrainingPath = 'TrainingDataset.mat'
TestingPath = 'TestingDataset.mat'

training_path = os.path.join(LoadingPath, TrainingPath)
testing_path = os.path.join(LoadingPath, TestingPath)

#### TrainingDataset

In [6]:
f_TrainingDataset = scipy.io.loadmat(training_path)
TrainingDataset = f_TrainingDataset['TrainingDataset'][0,SubjectID]

X_train_dataset = TrainingDataset['x']
Y_train_dataset = TrainingDataset['y']

nested_array_X = X_train_dataset[0][0]
X_train_dataset = np.array(nested_array_X)
X_train_dataset= np.transpose(X_train_dataset, (2, 1, 0))

nested_array_Y = Y_train_dataset[0][0]
Y_train_dataset = np.array(nested_array_Y)
Y_train_dataset=Y_train_dataset.flatten()

In [7]:
print(X_train_dataset.shape)
print(Y_train_dataset.shape)

(4239, 108, 50)
(4239,)


#### TestingDataset

In [8]:
f_TestingDataset = scipy.io.loadmat(testing_path)
TestingDataset = f_TestingDataset['TestingDataset'][0,SubjectID]

X_test_dataset = TestingDataset['x']
Y_test_dataset = TestingDataset['y']

nested_array_X = X_test_dataset[0][0]
X_test_dataset = np.array(nested_array_X)
X_test_dataset= np.transpose(X_test_dataset, (2, 1, 0))

nested_array_Y = Y_test_dataset[0][0]
Y_test_dataset = np.array(nested_array_Y)
Y_test_dataset=Y_test_dataset.flatten()

In [9]:
print(X_test_dataset.shape)
print(Y_test_dataset.shape)

(470, 108, 50)
(470,)


### Load : TrainingProjected & TestingProjected

In [12]:
TrainingProjectedPath = 'TrainingProjected.mat'
TestingProjectedPath = 'TestingProjected.mat'

trainingProjected_path = os.path.join(LoadingPath, TrainingProjectedPath)
testingProjected_path = os.path.join(LoadingPath, TestingProjectedPath)

#### TrainingProjected

In [13]:
f_TrainingProjected = scipy.io.loadmat(trainingProjected_path)
TrainingProjected = f_TrainingProjected['TrainingProjected'][0,SubjectID]

X_train_projected = TrainingProjected['x']
Y_train_projected = TrainingProjected['y']

nested_array_X = X_train_projected[0][0]
X_train_projected = np.array(nested_array_X)
X_train_projected= np.transpose(X_train_projected, (2, 0, 1))

nested_array_Y = Y_train_projected[0][0]
Y_train_projected = np.array(nested_array_Y)
Y_train_projected=Y_train_projected.flatten()

In [14]:
print(X_train_projected.shape)
print(Y_train_projected.shape)

(4239, 108, 50)
(4239,)


#### TestingProjected

In [15]:
f_TestingProjected = scipy.io.loadmat(testingProjected_path)
TestingProjected = f_TestingProjected['TestingProjected'][0,SubjectID]

X_test_projected = TestingProjected['x']
Y_test_projected = TestingProjected['y']

nested_array_X = X_test_projected[0][0]
X_test_projected = np.array(nested_array_X)
X_test_projected= np.transpose(X_test_projected, (2, 0, 1))

nested_array_Y = Y_test_projected[0][0]
Y_test_projected = np.array(nested_array_Y)
Y_test_projected=Y_test_projected.flatten()

In [16]:
print(X_test_projected.shape)
print(Y_test_projected.shape)

(470, 108, 50)
(470,)


### Load : TrainingFeatures & TestingFeatures

In [17]:
TrainingFeaturesPath = 'TrainingFeatures.mat'
TestingFeaturesPath = 'TestingFeatures.mat'

trainingFeatures_path = os.path.join(LoadingPath, TrainingFeaturesPath)
testingFeatures_path = os.path.join(LoadingPath, TestingFeaturesPath)

#### TrainingFeatures

In [18]:
f_TrainingFeatures = scipy.io.loadmat(trainingFeatures_path)

TrainingFeatures = f_TrainingFeatures['TrainingFeatures'][0, SubjectID]
X_train_features = TrainingFeatures[:, :108]

Y_train_features = TrainingFeatures[:, 108:]
Y_train_features = Y_train_features.flatten()

In [19]:
print(X_train_features.shape)
print(Y_train_features.shape)

(4239, 108)
(4239,)


#### TestingFeatures

In [20]:
f_TestingFeatures = scipy.io.loadmat(testingFeatures_path)

TestingFeatures = f_TestingFeatures['TestingFeatures'][0, SubjectID]
X_test_features = TestingFeatures[:, :108]

Y_test_features = TestingFeatures[:, 108:]
Y_test_features=Y_test_features.flatten()

In [21]:
print(X_test_features.shape)
print(Y_test_features.shape)

(470, 108)
(470,)


### Load: TrainingProjectedR & TestingProjectedR 
#### (If seperately found CSP & RCSP)

In [22]:
TrainingProjectedRPath = 'TrainingProjected_R.mat'
TestingProjectedRPath = 'TestingProjected_R.mat'

trainingProjectedR_path = os.path.join(LoadingPath, TrainingProjectedRPath)
testingProjectedR_path = os.path.join(LoadingPath, TestingProjectedRPath)

In [23]:
f_TrainingProjectedR = scipy.io.loadmat(trainingProjectedR_path)
TrainingProjectedR = f_TrainingProjectedR['TrainingProjected_R'][0,SubjectID]

X_train_projectedR = TrainingProjectedR['x']
Y_train_projectedR = TrainingProjectedR['y']

nested_array_X = X_train_projectedR[0][0]
X_train_projectedR = np.array(nested_array_X)
X_train_projectedR= np.transpose(X_train_projectedR, (2, 0, 1))

nested_array_Y = Y_train_projectedR[0][0]
Y_train_projectedR = np.array(nested_array_Y)
Y_train_projectedR=Y_train_projectedR.flatten()

In [24]:
f_TestingProjectedR = scipy.io.loadmat(testingProjectedR_path)
TestingProjectedR = f_TestingProjectedR['TestingProjected_R'][0,SubjectID]

X_test_projectedR = TestingProjectedR['x']
Y_test_projectedR = TestingProjectedR['y']

nested_array_X = X_test_projectedR[0][0]
X_test_projectedR = np.array(nested_array_X)
X_test_projectedR= np.transpose(X_test_projectedR, (2, 0, 1))

nested_array_Y = Y_test_projectedR[0][0]
Y_test_projectedR = np.array(nested_array_Y)
Y_test_projectedR=Y_test_projectedR.flatten()

In [25]:
print(X_train_projectedR.shape)
print(X_test_projectedR.shape)

(4239, 108, 50)
(470, 108, 50)


### Load covariance matrices (estimated in MATLAB) 

TrainingCovPath = 'TrainingCovarianceMatrices.mat'
TestingCovPath = 'TestingCovarianceMatrices.mat'

trainingCov_path = os.path.join(LoadingPath, TrainingCovPath)
testingCov_path = os.path.join(LoadingPath, TestingCovPath)

f_TrainingCov=h5py.File(trainingCov_path)

TrainingCov= f_TrainingCov['TrainingCovarianceMatrices']

Subject = TrainingCov[SubjectID, 0] 

TrainingCovMat = f_TrainingCov[sub_ref]

TrainingCovMat=np.array(TrainingCovMat)

TrainingCovMat.shape

f_TestingCov = scipy.io.loadmat(testingCov_path)
TestingCov = f_TestingCov ['TestingCovarianceMatrices'][0,SubjectID]

TestingCov=np.transpose(TestingCov, (2, 0, 1))

TestingCov.shape

TestingCovMat.shape

# Classification algorithms 

## "Riemannian geometry" (W/out CSP) 

In [26]:
cov_estimator = Covariances(estimator='lwf') # helps with regularisation 

X_train_cov=cov_estimator.transform(X_train_dataset)
# Apply learned transformation to testing dataset 
X_test_cov = cov_estimator.transform(X_test_dataset)

In [27]:
print(X_train_cov.shape)
print(X_test_cov.shape)

(4239, 108, 108)
(470, 108, 108)


In [28]:
clf_TS = TSclassifier(metric='riemann', clf=SVC(C=1.0, kernel='linear'))
clf_TS.fit(X_train_cov, Y_train_dataset)

# Make predictions on the test set
y_pred = clf_TS.predict(X_test_cov)

In [29]:
#Accuracy
accuracy = accuracy_score(Y_test_dataset, y_pred)
print(f"Accuracy: {accuracy * 100:.5f}%")

# Recall
recall = recall_score(Y_test_dataset, y_pred)  
print(f"Recall: {recall * 100:.5f}%")

# Precision
precision = precision_score(Y_test_dataset, y_pred)  
print(f"Precision: {precision * 100:.5f}%")

# F1 Score
f1 = f1_score(Y_test_dataset, y_pred)  
print(f"F1 Score: {f1 * 100:.5f}%")

Accuracy: 83.61702%
Recall: 83.40611%
Precision: 83.04348%
F1 Score: 83.22440%


## CSP: Arithmetic average "Spatial filtered (A) + Riemann"
### Project spatially filtered signals (ProjectedDataset) to tangent space 

In [30]:
# Find covariance matrix of X_train_projected
cov_estimator_projected = Covariances(estimator='lwf') # helps with regularisation 

X_train_cov_projected=cov_estimator_projected.transform(X_train_projected)
# Apply learned transformation to testing dataset 
X_test_cov_projected = cov_estimator_projected.transform(X_test_projected)

In [31]:
print(X_train_cov_projected.shape)
print(X_test_cov_projected.shape)

(4239, 108, 108)
(470, 108, 108)


In [32]:
clf_TS_proj = TSclassifier(metric='riemann', clf=SVC(C=1.0, kernel='linear'))
clf_TS_proj.fit(X_train_cov_projected, Y_train_projected)

# Make predictions on the test set
y_pred_proj = clf_TS_proj.predict(X_test_cov_projected)

In [33]:
#Accuracy
accuracy = accuracy_score(Y_test_dataset, y_pred_proj)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Recall
recall = recall_score(Y_test_dataset, y_pred_proj)  
print(f"Recall: {recall * 100:.2f}%")

# Precision
precision = precision_score(Y_test_dataset, y_pred_proj)  
print(f"Precision: {precision * 100:.2f}%")

# F1 Score
f1 = f1_score(Y_test_dataset, y_pred_proj)  
print(f"F1 Score: {f1 * 100:.2f}%")

Accuracy: 90.85%
Recall: 89.08%
Precision: 91.89%
F1 Score: 90.47%


## CSP: Arithmetic average "Spatial filtered (RCSP) (A) + Riemann"
### Project spatially filtered signals (ProjectedDataset_R) to tangent space 

In [34]:
# Find covariance matrix of X_train_projected
cov_estimator_projected = Covariances(estimator='lwf') # helps with regularisation 

X_train_cov_projectedR=cov_estimator_projected.transform(X_train_projectedR)
# Apply learned transformation to testing dataset 
X_test_cov_projectedR = cov_estimator_projected.transform(X_test_projectedR)

In [35]:
clf_TS_projR = TSclassifier(metric='riemann', clf=SVC(C=1.0, kernel='linear'))
clf_TS_projR.fit(X_train_cov_projectedR, Y_train_projectedR)

# Make predictions on the test set
y_pred_projR = clf_TS_proj.predict(X_test_cov_projectedR)

In [36]:
#Accuracy
accuracy = accuracy_score(Y_test_dataset, y_pred_projR)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Recall
recall = recall_score(Y_test_dataset, y_pred_projR)  
print(f"Recall: {recall * 100:.2f}%")

# Precision
precision = precision_score(Y_test_dataset, y_pred_projR)  
print(f"Precision: {precision * 100:.2f}%")

# F1 Score
f1 = f1_score(Y_test_dataset, y_pred_projR)  
print(f"F1 Score: {f1 * 100:.2f}%")

Accuracy: 60.43%
Recall: 89.96%
Precision: 55.83%
F1 Score: 68.90%


## CSP: Riemannian average (1) "CSP(R) + Riemannian"
### Use pyriemann.spatialfilters.csp (TrainingDataset) to obtain spatially filtered signals, project and classify in tangent space

In [37]:
csp = CSP(nfilter=n_components, metric="riemann", log=False)
cov_estimator = Covariances(estimator='lwf')

X_train_cov=cov_estimator.transform(X_train_dataset)
X_test_cov = cov_estimator.transform(X_test_dataset)

#csp.fit_transform takes covariance matrix as input 
X_train_csp = csp.fit_transform(X_train_cov, Y_train_dataset)

# Apply learned transformation to testing dataset 
X_test_csp = csp.transform(X_test_cov)

In [38]:
print(X_train_csp.shape)

(4239, 108, 108)


In [39]:
clf_TS_features = TSclassifier(metric='riemann', clf=SVC(C=1.0, kernel='linear'))
clf_TS_features.fit(X_train_csp, Y_train_dataset)

# Make predictions on the test set
y_pred_features = clf_TS_features.predict(X_test_csp)

In [40]:
#Accuracy
accuracy = accuracy_score(Y_test_dataset, y_pred_features)
print(f"Accuracy: {accuracy * 100:.5f}%")

# Recall
recall = recall_score(Y_test_dataset, y_pred_features)  
print(f"Recall: {recall * 100:.5f}%")

# Precision
precision = precision_score(Y_test_dataset, y_pred_features)  
print(f"Precision: {precision * 100:.5f}%")

# F1 Score
f1 = f1_score(Y_test_dataset, y_pred_features)  
print(f"F1 Score: {f1 * 100:.5f}%")

Accuracy: 83.61702%
Recall: 83.40611%
Precision: 83.04348%
F1 Score: 83.22440%


## CSP: Riemannian average (2) "CSP(R)"
### Classification w/out projection in TS

In [41]:
csp2 = CSP(nfilter=n_components, metric="riemann", log=True)
cov_estimator = Covariances(estimator='lwf')

X_train_cov=cov_estimator.transform(X_train_dataset)
X_test_cov = cov_estimator.transform(X_test_dataset)

#csp.fit_transform takes covariance matrix as input 
X_train_csp = csp2.fit_transform(X_train_cov, Y_train_dataset)

# Apply learned transformation to testing dataset 
X_test_csp = csp2.transform(X_test_cov)

In [42]:
print(X_train_csp.shape)

(4239, 108)


In [43]:
clf_csp = SVC(C=1.0, kernel='linear')
clf_csp.fit(X_train_csp, Y_train_dataset)

# Make predictions on the test set
y_pred_csp2 = clf_csp.predict(X_test_csp)

In [44]:
#Accuracy
accuracy = accuracy_score(Y_test_dataset, y_pred_csp2)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Recall
recall = recall_score(Y_test_dataset, y_pred_csp2)  
print(f"Recall: {recall * 100:.2f}%")

# Precision
precision = precision_score(Y_test_dataset, y_pred_csp2)  
print(f"Precision: {precision * 100:.2f}%")

# F1 Score
f1 = f1_score(Y_test_dataset, y_pred_csp2)  
print(f"F1 Score: {f1 * 100:.2f}%")

Accuracy: 71.70%
Recall: 73.80%
Precision: 69.83%
F1 Score: 71.76%


## CSP: "Concatenation" of features 
### Concatenate CSP variance features and tangent space features 

##### Find log-variance of spatially filtered matrices 

In [45]:
# loaded as X_train_features 
print(X_train_features.shape)

(4239, 108)


##### Find Tangent features 

In [46]:
cov_estimator = Covariances(estimator='lwf') # helps with regularisation 

X_train_cov=cov_estimator.transform(X_train_dataset)
# Apply learned transformation to testing dataset 
X_test_cov = cov_estimator.transform(X_test_dataset)

In [47]:
TSpace = TangentSpace(metric='riemann', tsupdate=False).fit(X_train_cov)
X_train_TS = TSpace.transform(X_train_cov)

X_test_TS = TSpace.transform(X_test_cov)

In [48]:
print(X_train_TS.shape) 

(4239, 5886)


##### Concatenate datasets 

In [49]:
X_train_cat=np.concatenate((X_train_features, X_train_TS), axis=1)

X_test_cat=np.concatenate((X_test_features, X_test_TS), axis=1)

##### Classify

In [50]:
clf_TS_cat = SVC(C=1.0, kernel='linear')
clf_TS_cat.fit(X_train_cat, Y_train_dataset)

# Make predictions on the test set
y_pred_cat = clf_TS_cat.predict(X_test_cat)

In [51]:
#Accuracy
accuracy = accuracy_score(Y_test_dataset, y_pred_cat)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Recall
recall = recall_score(Y_test_dataset, y_pred_cat)  
print(f"Recall: {recall * 100:.2f}%")

# Precision
precision = precision_score(Y_test_dataset, y_pred_cat)  
print(f"Precision: {precision * 100:.2f}%")

# F1 Score
f1 = f1_score(Y_test_dataset, y_pred_cat)  
print(f"F1 Score: {f1 * 100:.2f}%")

Accuracy: 81.49%
Recall: 81.22%
Precision: 80.87%
F1 Score: 81.05%


In [52]:
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time:", elapsed_time/60, "minutes")

Elapsed time: 10.145559712251027 minutes


### Obtain Cov matrix (without optimisation - MATLAB) - using PyR to get CSP(R) - project

#from pyriemann.utils.test import is_pos_def
#is_positive_definite = is_pos_def(X_train_cov)

#print(f"Is the matrix positive definite? {is_positive_definite}")

#csp = CSP(nfilter=n_components, metric="riemann", log=False)

#csp.fit_transform takes covariance matrix as input 
#X_train_csp = csp.fit_transform(TrainingCovMat, Y_train_dataset)

# Apply learned transformation to testing dataset 
#X_test_csp = csp.transform(TestingCovMat)

#clf_TS_features = TSclassifier(metric='riemann', clf=SVC(C=1.0, kernel='linear'))
#clf_TS_features.fit(X_train_csp, Y_train_dataset)

# Make predictions on the test set
#y_pred_features = clf_TS_features.predict(X_test_csp)